## Installing and importing libraries

In [ ]:
#Prerequisites (external libraries not found in Google Colab)
!pip install wikipedia
!pip install wikipedia-api
###Baseline classifiers libraries###

#Keras Neural Network (Feedforward)
from keras.metrics import top_k_categorical_accuracy
from keras.backend import clear_session
from keras.models import Sequential
from keras.layers import Dense
#Computations 
import numpy as np
import random
#Tables
import pandas as pd
from google.colab.data_table import DataTable
#Sklearn classifiers and utils
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split

  Created wheel for wikipedia: filename=wikipedia-1.4.0-cp36-none-any.whl size=11686 sha256=1786e0317d4163ff28b088299f002590172524d5d5fdb1d0477ed93d915e5d00
  Stored in directory: /root/.cache/pip/wheels/87/2a/18/4e471fd96d12114d16fe4a446d00c3b38fb9efcb744bd31f4a
Successfully built wikipedia
  Created wheel for wikipedia-api: filename=Wikipedia_API-0.5.4-cp36-none-any.whl size=13462 sha256=b3a51303dfaa73b7579af6ef6728c9f2a4f2932cd8f783a5c57c50af11db43ac
  Stored in directory: /root/.cache/pip/wheels/bf/40/42/ba1d497f3712281b659dd65b566fc868035c859239571a725a
Successfully built wikipedia-api


Using TensorFlow backend.


### Project library modules

In [1]:
!git clone https://github.com/andresC98/One-Shot-DocClassifier.git
%cd One-Shot-DocClassifier/lib
!ls

Cloning into 'One-Shot-DocClassifier'...
remote: Enumerating objects: 285, done.
remote: Counting objects: 100% (285/285), done.
remote: Compressing objects: 100% (208/208), done.
remote: Total 285 (delta 155), reused 183 (delta 75), pack-reused 0
Receiving objects: 100% (285/285), 1.94 MiB | 7.40 MiB/s, done.
Resolving deltas: 100% (155/155), done.
/content/One-Shot-DocClassifier/lib
arxiv_parser.py  doc_utils.py  max_sim_classifier.py  wiki_parser.py


In [ ]:
###Importing project library###
import doc_utils #cleaning and other helper functions (visualization...)
from max_sim_classifier import MaxSimClassifier #maximum similarity classifier
#Parser libraries: wikipedia dataset and arxiv dataset
from arxiv_parser import arxiv_parser
from wiki_parser import concurrentGetWikiFullPage, concurrentGetAllCatArticles

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


## Data retrieval by Parsers and preprocessing

In [ ]:
engineering_topic_defs = concurrentGetWikiFullPage(topics_list = doc_utils.ALL_TOPICS )

engineering_articles, n_test_samples = concurrentGetAllCatArticles(doc_utils.ALL_TOPICS, full_text_test=True)

Obtaining full wikipedia page for the topic: Chemical engineering. (Definition of Class #[0])Obtaining full wikipedia page for the topic: Biomedical engineering. (Definition of Class #[1])

Obtaining full wikipedia page for the topic: Civil engineering. (Definition of Class #[2])
Obtaining full wikipedia page for the topic: Electrical engineering. (Definition of Class #[3])Obtaining full wikipedia page for the topic: Mechanical engineering. (Definition of Class #[4])

Obtaining full wikipedia page for the topic: Aerospace engineering. (Definition of Class #[5])
Obtaining full wikipedia page for the topic: Software engineering. (Definition of Class #[6])
Obtaining full wikipedia page for the topic: Industrial engineering. (Definition of Class #[7])
Obtaining full wikipedia page for the topic: Computer engineering. (Definition of Class #[8])
Retrieved 38 articles from category topic 'Computer engineering'[TopicID:8]
Retrieved 57 articles from category topic 'Software engineering'[TopicID

In [ ]:
arxiv_topic_defs = concurrentGetWikiFullPage(topics_list = doc_utils.ARXIV_WIKI_TOPICS )

arxiv_dataset, paperslist = arxiv_parser(100)

Obtaining full wikipedia page for the topic: Computer science. (Definition of Class #[0])
Obtaining full wikipedia page for the topic: Economics. (Definition of Class #[1])
Obtaining full wikipedia page for the topic: Systems engineering. (Definition of Class #[2])
Obtaining full wikipedia page for the topic: Mathematics. (Definition of Class #[3])
Obtaining full wikipedia page for the topic: Astrophysics. (Definition of Class #[4])
Obtaining full wikipedia page for the topic: Computational biology. (Definition of Class #[5])
Obtaining full wikipedia page for the topic: Quantitative finance. (Definition of Class #[6])
Obtaining full wikipedia page for the topic: Statistics. (Definition of Class #[7])
Retrieving papers for subject: computer_science
Retrieving papers for subject: economics
Retrieving papers for subject: eess
Retrieving papers for subject: mathematics
Retrieving papers for subject: physics
Retrieving papers for subject: q_biology
Retrieving papers for subject: q_finance
R

In [ ]:
#MSC ,SVM and MNB compatible (clf_data): NOTE: Preprocess type selected in pipeline later
x_train_CLF_W_defs, y_train_CLF_W_defs, x_test_CLF_W_full, y_test_CLF_W_full = doc_utils.processClassifierData(engineering_topic_defs, engineering_articles,topics = doc_utils.ALL_TOPICS,dataset_type ="wiki")
x_train_CLF_A_defs, y_train_CLF_A_defs, x_test_CLF_A_full, y_test_CLF_A_full = doc_utils.processClassifierData(arxiv_topic_defs, arxiv_dataset,topics = doc_utils.ARXIV_WIKI_TOPICS,dataset_type ="arxiv")

In [ ]:
x_train_FFNN_W_C_defs, y_train_FFNN_W_C_defs, x_test_FFNN_W_C_full, y_test_FFNN_W_C_full, dictionary_FFNN_W_C = doc_utils.processNeuralNetData(engineering_topic_defs,engineering_articles,dataset_type = "wiki",preprocess = 'custom')
x_train_FFNN_A_C_defs, y_train_FFNN_A_C_defs, x_test_FFNN_A_C_full, y_test_FFNN_A_C_full, dictionary_FFNN_A_C = doc_utils.processNeuralNetData(arxiv_topic_defs,arxiv_dataset,dataset_type = "arxiv",preprocess = 'custom')

#Document classification using an article per category - Baseline models

## Support Vector Machine

### Wiki dataset

In [ ]:
top2_acc_list = list()
top1_acc_list = list()

for i in range(30):
    x_eng_articles = ["", "", "", "", "", "", "", "", ""]
    y_eng_labels = list()
    for category_arts in engineering_articles:
        n_articles = len(category_arts[0])
        category_class_id = category_arts[1]
        id = random.randint(0,n_articles-1)
        #print("Article #{} was chosen from topic {}".format(id, category_class_id))
        x_eng_articles[category_class_id] = category_arts[0][id]
        y_eng_labels.append(category_class_id)


    svm = Pipeline([('vect', CountVectorizer(stop_words='english',ngram_range=(1,1))),
                    ('tfidf', TfidfTransformer(use_idf=True)),
                    ('clf', SVC(C=0.01, gamma=0.01)),
                    ])
    svm.fit(x_eng_articles, y_eng_labels)

    y_test_preds_proba = svm.decision_function(x_test_CLF_W_full)
    top1_acc, top2_acc = doc_utils.top2_acc(y_test_preds_proba, y_test_CLF_W_full,verbose=0)
    
    print("Run #{}. SVM Top-1 Acc: {:.4f}, Top-2 Acc: {:.4f}".format(i, top1_acc, top2_acc))
    top2_acc_list.append(top2_acc)
    top1_acc_list.append(top1_acc)

print("\nAvg Top-2 acc:{:.4f}+-{:.4f}".format(np.mean(np.array(top2_acc_list)),np.std(np.array(top2_acc_list))))
print("Avg Top-1 acc:{:.4f}+-{:.4f}".format(np.mean(np.array(top1_acc_list)),np.std(np.array(top1_acc_list))))

Run #0. SVM Top-1 Acc: 0.3407, Top-2 Acc: 0.4844
Run #1. SVM Top-1 Acc: 0.3045, Top-2 Acc: 0.4533
Run #2. SVM Top-1 Acc: 0.2372, Top-2 Acc: 0.4101
Run #3. SVM Top-1 Acc: 0.3779, Top-2 Acc: 0.5457
Run #4. SVM Top-1 Acc: 0.3508, Top-2 Acc: 0.4814
Run #5. SVM Top-1 Acc: 0.3106, Top-2 Acc: 0.4774
Run #6. SVM Top-1 Acc: 0.3598, Top-2 Acc: 0.5065
Run #7. SVM Top-1 Acc: 0.3930, Top-2 Acc: 0.5859
Run #8. SVM Top-1 Acc: 0.3106, Top-2 Acc: 0.4281
Run #9. SVM Top-1 Acc: 0.3176, Top-2 Acc: 0.4894
Run #10. SVM Top-1 Acc: 0.3106, Top-2 Acc: 0.4663
Run #11. SVM Top-1 Acc: 0.3226, Top-2 Acc: 0.4854
Run #12. SVM Top-1 Acc: 0.3015, Top-2 Acc: 0.5146
Run #13. SVM Top-1 Acc: 0.3417, Top-2 Acc: 0.4985
Run #14. SVM Top-1 Acc: 0.3387, Top-2 Acc: 0.4734
Run #15. SVM Top-1 Acc: 0.3910, Top-2 Acc: 0.5246
Run #16. SVM Top-1 Acc: 0.3698, Top-2 Acc: 0.5749
Run #17. SVM Top-1 Acc: 0.3558, Top-2 Acc: 0.5106
Run #18. SVM Top-1 Acc: 0.3528, Top-2 Acc: 0.5206
Run #19. SVM Top-1 Acc: 0.2754, Top-2 Acc: 0.4955
Run #20. S

### arXiv dataset

In [ ]:
top2_acc_list = list()
top1_acc_list = list()

for i in range(30):
    x_arxiv_papers = ["", "", "", "", "", "", "", ""]
    y_arxiv_labels = list()
    for category in arxiv_dataset:
        n_papers = len(category['papers'])
        category_class_id = category['label']
        id = random.randint(0,n_articles-1)
        #print("Paper #{} was chosen from topic {}".format(id, category_class_id))
        x_arxiv_papers[category_class_id] = ". ".join([category['papers'][id]['title'], category['papers'][id]['abstract']])
        y_arxiv_labels.append(category_class_id)


    svm = Pipeline([('vect', CountVectorizer(stop_words='english',ngram_range=(1,1))),
                    ('tfidf', TfidfTransformer(use_idf=True)),
                    ('clf', SVC(C=0.01, gamma=0.01)),
                    ])
    svm.fit(x_arxiv_papers, y_arxiv_labels)

    y_test_preds_proba = svm.decision_function(x_test_CLF_A_full)
    top1_acc, top2_acc = doc_utils.top2_acc(y_test_preds_proba, y_test_CLF_A_full,verbose=0)
    
    print("Run #{}. SVM Top-1 Acc: {:.4f}, Top-2 Acc: {:.4f}".format(i, top1_acc, top2_acc))
    top2_acc_list.append(top2_acc)
    top1_acc_list.append(top1_acc)

print("\nAvg Top-2 acc:{:.4f}+-{:.4f}".format(np.mean(np.array(top2_acc_list)),np.std(np.array(top2_acc_list))))
print("Avg Top-1 acc:{:.4f}+-{:.4f}".format(np.mean(np.array(top1_acc_list)),np.std(np.array(top1_acc_list))))

Run #0. SVM Top-1 Acc: 0.2462, Top-2 Acc: 0.4263
Run #1. SVM Top-1 Acc: 0.2025, Top-2 Acc: 0.3625
Run #2. SVM Top-1 Acc: 0.2150, Top-2 Acc: 0.3738
Run #3. SVM Top-1 Acc: 0.2975, Top-2 Acc: 0.4913
Run #4. SVM Top-1 Acc: 0.2900, Top-2 Acc: 0.4275
Run #5. SVM Top-1 Acc: 0.2712, Top-2 Acc: 0.4213
Run #6. SVM Top-1 Acc: 0.1888, Top-2 Acc: 0.3212
Run #7. SVM Top-1 Acc: 0.2475, Top-2 Acc: 0.4025
Run #8. SVM Top-1 Acc: 0.2375, Top-2 Acc: 0.4037
Run #9. SVM Top-1 Acc: 0.1888, Top-2 Acc: 0.3663
Run #10. SVM Top-1 Acc: 0.2213, Top-2 Acc: 0.4062
Run #11. SVM Top-1 Acc: 0.3212, Top-2 Acc: 0.4688
Run #12. SVM Top-1 Acc: 0.2437, Top-2 Acc: 0.4113
Run #13. SVM Top-1 Acc: 0.2013, Top-2 Acc: 0.3650
Run #14. SVM Top-1 Acc: 0.2437, Top-2 Acc: 0.4037
Run #15. SVM Top-1 Acc: 0.2550, Top-2 Acc: 0.4113
Run #16. SVM Top-1 Acc: 0.2150, Top-2 Acc: 0.3625
Run #17. SVM Top-1 Acc: 0.2737, Top-2 Acc: 0.4350
Run #18. SVM Top-1 Acc: 0.3013, Top-2 Acc: 0.4475
Run #19. SVM Top-1 Acc: 0.2988, Top-2 Acc: 0.4500
Run #20. S

Avg Top-2 acc:0.4095+-0.0330
Avg Top-1 acc:0.2496+-0.0241

## Multinomial Naive Bayes

### Wiki dataset

In [ ]:
top2_acc_list = list()
top1_acc_list = list()

for i in range(30):
    x_eng_articles = ["", "", "", "", "", "", "", "", ""]
    y_eng_labels = list()
    for category_arts in engineering_articles:
        n_articles = len(category_arts[0])
        category_class_id = category_arts[1]
        id = random.randint(0,n_articles-1)
        #print("Article #{} was chosen from topic {}".format(id, category_class_id))
        x_eng_articles[category_class_id] = category_arts[0][id]
        y_eng_labels.append(category_class_id)


    mnb = Pipeline([('vect', CountVectorizer(stop_words='english',ngram_range=(1,2))),
                    ('tfidf', TfidfTransformer(use_idf=True)),
                    ('clf', MultinomialNB(alpha=0.01)),
                    ])    
    mnb.fit(x_eng_articles, y_eng_labels)

    y_test_preds_proba = mnb.predict_proba(x_test_CLF_W_full)
    top1_acc, top2_acc = doc_utils.top2_acc(y_test_preds_proba, y_test_CLF_W_full,verbose=0)
    
    print("Run #{}. MNB Top-1 Acc: {:.4f}, Top-2 Acc: {:.4f}".format(i, top1_acc, top2_acc))
    top2_acc_list.append(top2_acc)
    top1_acc_list.append(top1_acc)

print("\nAvg Top-2 acc:{:.4f}+-{:.4f}".format(np.mean(np.array(top2_acc_list)),np.std(np.array(top2_acc_list))))
print("Avg Top-1 acc:{:.4f}+-{:.4f}".format(np.mean(np.array(top1_acc_list)),np.std(np.array(top1_acc_list))))

Run #0. MNB Top-1 Acc: 0.3739, Top-2 Acc: 0.5548
Run #1. MNB Top-1 Acc: 0.2663, Top-2 Acc: 0.4181
Run #2. MNB Top-1 Acc: 0.3789, Top-2 Acc: 0.5337
Run #3. MNB Top-1 Acc: 0.2704, Top-2 Acc: 0.4312
Run #4. MNB Top-1 Acc: 0.2985, Top-2 Acc: 0.4975
Run #5. MNB Top-1 Acc: 0.2372, Top-2 Acc: 0.3578
Run #6. MNB Top-1 Acc: 0.3739, Top-2 Acc: 0.5407
Run #7. MNB Top-1 Acc: 0.4392, Top-2 Acc: 0.5799
Run #8. MNB Top-1 Acc: 0.3216, Top-2 Acc: 0.5005
Run #9. MNB Top-1 Acc: 0.3045, Top-2 Acc: 0.4854
Run #10. MNB Top-1 Acc: 0.4040, Top-2 Acc: 0.5859
Run #11. MNB Top-1 Acc: 0.2955, Top-2 Acc: 0.4844
Run #12. MNB Top-1 Acc: 0.2905, Top-2 Acc: 0.4784
Run #13. MNB Top-1 Acc: 0.3116, Top-2 Acc: 0.4583
Run #14. MNB Top-1 Acc: 0.3206, Top-2 Acc: 0.5085
Run #15. MNB Top-1 Acc: 0.3106, Top-2 Acc: 0.4583
Run #16. MNB Top-1 Acc: 0.3829, Top-2 Acc: 0.5779
Run #17. MNB Top-1 Acc: 0.3638, Top-2 Acc: 0.5307
Run #18. MNB Top-1 Acc: 0.2804, Top-2 Acc: 0.4503
Run #19. MNB Top-1 Acc: 0.3789, Top-2 Acc: 0.5467
Run #20. M

### arXiv dataset

In [ ]:
top2_acc_list = list()
top1_acc_list = list()

for i in range(30):
    x_arxiv_papers = ["", "", "", "", "", "", "", ""]
    y_arxiv_labels = list()
    for category in arxiv_dataset:
        n_papers = len(category['papers'])
        category_class_id = category['label']
        id = random.randint(0,n_articles-1)
        #print("Paper #{} was chosen from topic {}".format(id, category_class_id))
        x_arxiv_papers[category_class_id] = ". ".join([category['papers'][id]['title'], category['papers'][id]['abstract']])
        y_arxiv_labels.append(category_class_id)


    mnb = Pipeline([('vect', CountVectorizer(stop_words='english',ngram_range=(1,2))),
                    ('tfidf', TfidfTransformer(use_idf=True)),
                    ('clf', MultinomialNB(alpha=0.01)),
                    ]) 
    mnb.fit(x_arxiv_papers, y_arxiv_labels)

    y_test_preds_proba = mnb.predict_proba(x_test_CLF_A_full)
    top1_acc, top2_acc = doc_utils.top2_acc(y_test_preds_proba, y_test_CLF_A_full,verbose=0)
    
    print("Run #{}. MNB Top-1 Acc: {:.4f}, Top-2 Acc: {:.4f}".format(i, top1_acc, top2_acc))
    top2_acc_list.append(top2_acc)
    top1_acc_list.append(top1_acc)

print("\nAvg Top-2 acc:{:.4f}+-{:.4f}".format(np.mean(np.array(top2_acc_list)),np.std(np.array(top2_acc_list))))
print("Avg Top-1 acc:{:.4f}+-{:.4f}".format(np.mean(np.array(top1_acc_list)),np.std(np.array(top1_acc_list))))

Run #0. MNB Top-1 Acc: 0.2375, Top-2 Acc: 0.4213
Run #1. MNB Top-1 Acc: 0.2938, Top-2 Acc: 0.4675
Run #2. MNB Top-1 Acc: 0.2975, Top-2 Acc: 0.4650
Run #3. MNB Top-1 Acc: 0.2625, Top-2 Acc: 0.4075
Run #4. MNB Top-1 Acc: 0.2375, Top-2 Acc: 0.3775
Run #5. MNB Top-1 Acc: 0.2550, Top-2 Acc: 0.4150
Run #6. MNB Top-1 Acc: 0.2300, Top-2 Acc: 0.3812
Run #7. MNB Top-1 Acc: 0.2600, Top-2 Acc: 0.4437
Run #8. MNB Top-1 Acc: 0.2637, Top-2 Acc: 0.4025
Run #9. MNB Top-1 Acc: 0.2737, Top-2 Acc: 0.4250
Run #10. MNB Top-1 Acc: 0.2587, Top-2 Acc: 0.4138
Run #11. MNB Top-1 Acc: 0.2863, Top-2 Acc: 0.4313
Run #12. MNB Top-1 Acc: 0.2675, Top-2 Acc: 0.4363
Run #13. MNB Top-1 Acc: 0.2537, Top-2 Acc: 0.4100
Run #14. MNB Top-1 Acc: 0.2988, Top-2 Acc: 0.4713
Run #15. MNB Top-1 Acc: 0.1963, Top-2 Acc: 0.3475
Run #16. MNB Top-1 Acc: 0.2288, Top-2 Acc: 0.3688
Run #17. MNB Top-1 Acc: 0.2800, Top-2 Acc: 0.4600
Run #18. MNB Top-1 Acc: 0.2100, Top-2 Acc: 0.3475
Run #19. MNB Top-1 Acc: 0.2712, Top-2 Acc: 0.4537
Run #20. M

## Feed Forward Neural Network

In [ ]:
from keras.utils import to_categorical
#Defining TOP-2 accuracy 
import functools
top2_accuracy = functools.partial(top_k_categorical_accuracy, k=2)
top2_accuracy.__name__ = 'TOP2_ACC'

### Wiki dataset

In [ ]:
top2_acc_list = list()
top1_acc_list = list()
cummulative_id = 0
for i in range(30):
    x_eng_articles = ["", "", "", "", "", "", "", "", ""]
    y_eng_labels = list()
    cummulative_id = 0

    for category_arts in engineering_articles:
        n_articles = len(category_arts[0])
        category_class_id = category_arts[1]
        id = random.randint(0,n_articles-1)
        #print("Article #{} was chosen from topic {}".format(id, category_class_id))
        x_eng_articles[category_class_id] = x_test_FFNN_W_C_full[cummulative_id + id]
        y_eng_labels.append(y_test_FFNN_W_C_full[cummulative_id + id])
        cummulative_id += n_articles

    x_eng_articles = np.stack( x_eng_articles )
    y_eng_labels = np.stack(y_eng_labels)
    #y_eng_labels = to_categorical(y_eng_labels)

    clear_session()
    
    ffnn_model = Sequential()
    ffnn_model.add(Dense(64, activation='relu', input_shape=(len(dictionary_FFNN_W_C),)))
    ffnn_model.add(Dense(len(doc_utils.ALL_TOPICS), activation='softmax'))
    ffnn_model.compile(optimizer='rmsprop', loss='categorical_crossentropy',metrics=['accuracy', top2_accuracy])
    
    hist = ffnn_model.fit(x_eng_articles, y_eng_labels, epochs=1,verbose=0)
    
    model_top1_acc = ffnn_model.evaluate(x_test_FFNN_W_C_full, y_test_FFNN_W_C_full, verbose=0)[1]
    model_top2_acc = ffnn_model.evaluate(x_test_FFNN_W_C_full, y_test_FFNN_W_C_full, verbose=0)[2]
    
    print("Run #{}. FFNN Top-1 Acc: {:.7f}, Top-2 Acc: {:.7f}".format(i, model_top1_acc, model_top2_acc))
    top2_acc_list.append(model_top2_acc)
    top1_acc_list.append(model_top1_acc)

print("\nAvg Top-2 acc:{:.7f}+-{:.7f}".format(np.mean(np.array(top2_acc_list)),np.std(np.array(top2_acc_list))))
print("Avg Top-1 acc:{:.7f}+-{:.7f}".format(np.mean(np.array(top1_acc_list)),np.std(np.array(top1_acc_list))))

Run #0. FFNN Top-1 Acc: 0.2512563, Top-2 Acc: 0.4130653
Run #1. FFNN Top-1 Acc: 0.1095477, Top-2 Acc: 0.3135678
Run #2. FFNN Top-1 Acc: 0.0984925, Top-2 Acc: 0.2000000
Run #3. FFNN Top-1 Acc: 0.1326633, Top-2 Acc: 0.2683417
Run #4. FFNN Top-1 Acc: 0.2201005, Top-2 Acc: 0.3115578
Run #5. FFNN Top-1 Acc: 0.1557789, Top-2 Acc: 0.2231156
Run #6. FFNN Top-1 Acc: 0.1537688, Top-2 Acc: 0.2080402
Run #7. FFNN Top-1 Acc: 0.1638191, Top-2 Acc: 0.2984925
Run #8. FFNN Top-1 Acc: 0.1698492, Top-2 Acc: 0.2964824
Run #9. FFNN Top-1 Acc: 0.2100503, Top-2 Acc: 0.3025126
Run #10. FFNN Top-1 Acc: 0.1236181, Top-2 Acc: 0.2301508
Run #11. FFNN Top-1 Acc: 0.1427136, Top-2 Acc: 0.2422111
Run #12. FFNN Top-1 Acc: 0.2241206, Top-2 Acc: 0.3276382
Run #13. FFNN Top-1 Acc: 0.2030151, Top-2 Acc: 0.3015075
Run #14. FFNN Top-1 Acc: 0.2261306, Top-2 Acc: 0.3356784
Run #15. FFNN Top-1 Acc: 0.1085427, Top-2 Acc: 0.2472362
Run #16. FFNN Top-1 Acc: 0.0703518, Top-2 Acc: 0.1497487
Run #17. FFNN Top-1 Acc: 0.1396985, Top-2

In [ ]:
top2_acc_list = list()
top1_acc_list = list()
cummulative_id = 0
for i in range(30):
    x_eng_articles = ["", "", "", "", "", "", "", "", ""]
    y_eng_labels = list()
    cummulative_id = 0

    for category_arts in engineering_articles:
        n_articles = len(category_arts[0])
        category_class_id = category_arts[1]
        id = random.randint(0,n_articles-1)
        #print("Article #{} was chosen from topic {}".format(id, category_class_id))
        x_eng_articles[category_class_id] = x_test_FFNN_W_C_full[cummulative_id + id]
        y_eng_labels.append(y_test_FFNN_W_C_full[cummulative_id + id])
        cummulative_id += n_articles

    x_eng_articles = np.stack( x_eng_articles )
    y_eng_labels = np.stack(y_eng_labels)
    #y_eng_labels = to_categorical(y_eng_labels)

    clear_session()
    
    ffnn_model = Sequential()
    ffnn_model.add(Dense(256, activation='relu', input_shape=(len(dictionary_FFNN_W_C),)))
    ffnn_model.add(Dense(len(doc_utils.ALL_TOPICS), activation='softmax'))
    ffnn_model.compile(optimizer='rmsprop', loss='categorical_crossentropy',metrics=['accuracy', top2_accuracy])
    
    hist = ffnn_model.fit(x_eng_articles, y_eng_labels, epochs=5,verbose=0)
    
    model_top1_acc = ffnn_model.evaluate(x_test_FFNN_W_C_full, y_test_FFNN_W_C_full, verbose=0)[1]
    model_top2_acc = ffnn_model.evaluate(x_test_FFNN_W_C_full, y_test_FFNN_W_C_full, verbose=0)[2]
    
    print("Run #{}. FFNN Top-1 Acc: {:.7f}, Top-2 Acc: {:.7f}".format(i, model_top1_acc, model_top2_acc))
    top2_acc_list.append(model_top2_acc)
    top1_acc_list.append(model_top1_acc)

print("\nAvg Top-2 acc:{:.7f}+-{:.7f}".format(np.mean(np.array(top2_acc_list)),np.std(np.array(top2_acc_list))))
print("Avg Top-1 acc:{:.7f}+-{:.7f}".format(np.mean(np.array(top1_acc_list)),np.std(np.array(top1_acc_list))))

Run #0. FFNN Top-1 Acc: 0.2502513, Top-2 Acc: 0.4412060
Run #1. FFNN Top-1 Acc: 0.2442211, Top-2 Acc: 0.3829146
Run #2. FFNN Top-1 Acc: 0.2211055, Top-2 Acc: 0.3396985
Run #3. FFNN Top-1 Acc: 0.1135678, Top-2 Acc: 0.3336684
Run #4. FFNN Top-1 Acc: 0.2291457, Top-2 Acc: 0.3889447
Run #5. FFNN Top-1 Acc: 0.2814070, Top-2 Acc: 0.3939699
Run #6. FFNN Top-1 Acc: 0.2050251, Top-2 Acc: 0.3527638
Run #7. FFNN Top-1 Acc: 0.1266332, Top-2 Acc: 0.2914573
Run #8. FFNN Top-1 Acc: 0.2562814, Top-2 Acc: 0.4251256
Run #9. FFNN Top-1 Acc: 0.1758794, Top-2 Acc: 0.2944724
Run #10. FFNN Top-1 Acc: 0.2884422, Top-2 Acc: 0.4140703
Run #11. FFNN Top-1 Acc: 0.0613065, Top-2 Acc: 0.2251256
Run #12. FFNN Top-1 Acc: 0.2130653, Top-2 Acc: 0.2974874
Run #13. FFNN Top-1 Acc: 0.1427136, Top-2 Acc: 0.2683417
Run #14. FFNN Top-1 Acc: 0.2030151, Top-2 Acc: 0.3055276
Run #15. FFNN Top-1 Acc: 0.1628141, Top-2 Acc: 0.2874372
Run #16. FFNN Top-1 Acc: 0.2120603, Top-2 Acc: 0.3949749
Run #17. FFNN Top-1 Acc: 0.1889447, Top-2

### arXiv dataset

In [ ]:
top2_acc_list = list()
top1_acc_list = list()
for i in range(30):
    cummulative_id = 0
    x_arxiv_papers = ["", "", "", "", "", "", "", ""]
    y_arxiv_labels = list()
    for category in arxiv_dataset:
        n_papers = len(category['papers'])
        category_class_id = category['label']
        id = random.randint(0,n_articles-1)
        #print("Article #{} was chosen from topic {}".format(id, category_class_id))
        x_arxiv_papers[category_class_id] = x_test_FFNN_A_C_full[cummulative_id + id]
        y_arxiv_labels.append(y_test_FFNN_A_C_full[cummulative_id + id])
        cummulative_id += n_articles

    x_arxiv_papers = np.stack( x_arxiv_papers )
    y_arxiv_labels = np.stack(y_arxiv_labels)

    clear_session()
    
    ffnn_model = Sequential()
    ffnn_model.add(Dense(256, activation='relu', input_shape=(len(dictionary_FFNN_A_C),)))
    ffnn_model.add(Dense(len(doc_utils.ARXIV_WIKI_TOPICS), activation='softmax'))
    ffnn_model.compile(optimizer='rmsprop', loss='categorical_crossentropy',metrics=['accuracy', top2_accuracy])
    
    hist = ffnn_model.fit(x_arxiv_papers, y_arxiv_labels, epochs=5,verbose=0)
    
    model_top1_acc = ffnn_model.evaluate(x_test_FFNN_A_C_full, y_test_FFNN_A_C_full, verbose=0)[1]
    model_top2_acc = ffnn_model.evaluate(x_test_FFNN_A_C_full, y_test_FFNN_A_C_full, verbose=0)[2]
    
    print("Run #{}. FFNN Top-1 Acc: {:.7f}, Top-2 Acc: {:.7f}".format(i, model_top1_acc, model_top2_acc))
    top2_acc_list.append(model_top2_acc)
    top1_acc_list.append(model_top1_acc)

print("\nAvg Top-2 acc:{:.7f}+-{:.7f}".format(np.mean(np.array(top2_acc_list)),np.std(np.array(top2_acc_list))))
print("Avg Top-1 acc:{:.7f}+-{:.7f}".format(np.mean(np.array(top1_acc_list)),np.std(np.array(top1_acc_list))))

Run #0. FFNN Top-1 Acc: 0.1437500, Top-2 Acc: 0.2875000
Run #1. FFNN Top-1 Acc: 0.1575000, Top-2 Acc: 0.2712500
Run #2. FFNN Top-1 Acc: 0.1400000, Top-2 Acc: 0.2562500
Run #3. FFNN Top-1 Acc: 0.1662500, Top-2 Acc: 0.2900000
Run #4. FFNN Top-1 Acc: 0.1750000, Top-2 Acc: 0.2900000
Run #5. FFNN Top-1 Acc: 0.1687500, Top-2 Acc: 0.2825000
Run #6. FFNN Top-1 Acc: 0.1825000, Top-2 Acc: 0.3087500
Run #7. FFNN Top-1 Acc: 0.1712500, Top-2 Acc: 0.2862500
Run #8. FFNN Top-1 Acc: 0.1487500, Top-2 Acc: 0.2800000
Run #9. FFNN Top-1 Acc: 0.1812500, Top-2 Acc: 0.2700000
Run #10. FFNN Top-1 Acc: 0.1587500, Top-2 Acc: 0.2825000
Run #11. FFNN Top-1 Acc: 0.1600000, Top-2 Acc: 0.3025000
Run #12. FFNN Top-1 Acc: 0.1837500, Top-2 Acc: 0.2675000
Run #13. FFNN Top-1 Acc: 0.1700000, Top-2 Acc: 0.2662500
Run #14. FFNN Top-1 Acc: 0.1837500, Top-2 Acc: 0.2712500
Run #15. FFNN Top-1 Acc: 0.1737500, Top-2 Acc: 0.2987500
Run #16. FFNN Top-1 Acc: 0.1775000, Top-2 Acc: 0.2712500
Run #17. FFNN Top-1 Acc: 0.1825000, Top-2

#Document classification using an article per category - MSC with document embeddings

## Maximum Similarity Classifier

### Wiki dataset

In [ ]:
#Accounting for stochasticity of the method
best_hparams = {'preprocess': 'custom', 'dm': 0, 'epochs': 50, 'vector_size': 50, 'min_count': 1, 'window': 3}
top2_acc_list = list()
top1_acc_list = list()

for i in range(30):
    x_eng_articles = ["", "", "", "", "", "", "", "", ""]
    y_eng_labels = list()
    for category_arts in engineering_articles:
        n_articles = len(category_arts[0])
        category_class_id = category_arts[1]
        id = random.randint(0,n_articles-1)
        #print("Article #{} was chosen from topic {}".format(id, category_class_id))
        x_eng_articles[category_class_id] = category_arts[0][id]
        y_eng_labels.append(category_class_id)
    
    
    max_sim_model = MaxSimClassifier("wiki", best_hparams['preprocess'], best_hparams['vector_size'],best_hparams['min_count'],
                                    best_hparams['epochs'],best_hparams['dm'],best_hparams['window'],workers=8)

    max_sim_model.fit_articles(x_eng_articles, y_eng_labels)

    top2_acc = max_sim_model.score(x_test_CLF_W_full, y_test_CLF_W_full,eval="top2") 
    top1_acc = max_sim_model.score(x_test_CLF_W_full, y_test_CLF_W_full,eval="top1")

    print("Run {}. MSC Top-1 Acc: {:.4f}, Top-2 Acc.:{:.4f}".format(i, top1_acc, top2_acc))
    top2_acc_list.append(top2_acc)
    top1_acc_list.append(top1_acc)


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


Run 0. MSC Top-1 Acc: 0.3226, Top-2 Acc.:0.5196
Run 1. MSC Top-1 Acc: 0.3085, Top-2 Acc.:0.4834
Run 2. MSC Top-1 Acc: 0.3568, Top-2 Acc.:0.5166
Run 3. MSC Top-1 Acc: 0.1889, Top-2 Acc.:0.3397
Run 4. MSC Top-1 Acc: 0.3176, Top-2 Acc.:0.5035
Run 5. MSC Top-1 Acc: 0.3216, Top-2 Acc.:0.4764
Run 6. MSC Top-1 Acc: 0.3206, Top-2 Acc.:0.4372
Run 7. MSC Top-1 Acc: 0.3849, Top-2 Acc.:0.5457
Run 8. MSC Top-1 Acc: 0.3085, Top-2 Acc.:0.4563
Run 9. MSC Top-1 Acc: 0.2523, Top-2 Acc.:0.4000
Run 10. MSC Top-1 Acc: 0.2874, Top-2 Acc.:0.4482
Run 11. MSC Top-1 Acc: 0.3578, Top-2 Acc.:0.5367
Run 12. MSC Top-1 Acc: 0.2754, Top-2 Acc.:0.4030
Run 13. MSC Top-1 Acc: 0.2724, Top-2 Acc.:0.4221
Run 14. MSC Top-1 Acc: 0.3236, Top-2 Acc.:0.4945
Run 15. MSC Top-1 Acc: 0.3246, Top-2 Acc.:0.4864
Run 16. MSC Top-1 Acc: 0.3106, Top-2 Acc.:0.4653
Run 17. MSC Top-1 Acc: 0.2201, Top-2 Acc.:0.3477
Run 18. MSC Top-1 Acc: 0.3126, Top-2 Acc.:0.4573
Run 19. MSC Top-1 Acc: 0.3889, Top-2 Acc.:0.5538
Run 20. MSC Top-1 Acc: 0.3508,

In [ ]:
print("Avg Top-2 acc:{:.4f}+-{:.4f}".format(np.mean(np.array(top2_acc_list)),np.std(np.array(top2_acc_list))))
print("Avg Top-1 acc:{:.4f}+-{:.4f}".format(np.mean(np.array(top1_acc_list)),np.std(np.array(top1_acc_list))))

Avg Top-2 acc:0.4777+-0.0618
Avg Top-1 acc:0.3173+-0.0506


In [ ]:
x_eng_articles

['The Deal–Grove model mathematically describes the growth of an oxide layer on the surface of a material.  In particular, it is used to predict and interpret thermal oxidation of silicon in semiconductor device fabrication. The model was first published in 1965 by Bruce Deal and Andrew Grove of Fairchild Semiconductor, building on Mohamed M. Atalla\'s work on silicon surface passivation by thermal oxidation at Bell Labs in the late 1950s. This served as a step in the development of CMOS devices and the fabrication of integrated circuits.\n\nPhysical assumptions\nThe model assumes that oxidation reaction occurs at the interface between the oxide layer and the substrate material, rather than between the oxide and the ambient gas.  Thus, it considers three phenomena that the oxidizing species undergoes, in this order:\n\nIt diffuses from the bulk of the ambient gas to the surface.\nIt diffuses through the existing oxide layer to the oxide-substrate interface.\nIt reacts with the substrat

### arXiv dataset

In [ ]:
#Accounting for stochasticity of the method
best_hparams = {'preprocess': 'custom', 'dm': 0, 'epochs': 50, 'vector_size': 50, 'min_count': 1, 'window': 3}
top2_acc_list = list()
top1_acc_list = list()

for i in range(30):
    x_arxiv_papers = ["", "", "", "", "", "", "", ""]
    y_arxiv_labels = list()
    for category in arxiv_dataset:
        n_papers = len(category['papers'])
        category_class_id = category['label']
        id = random.randint(0,n_articles-1)
        #print("Paper #{} was chosen from topic {}".format(id, category_class_id))
        x_arxiv_papers[category_class_id] = ". ".join([category['papers'][id]['title'], category['papers'][id]['abstract']])
        y_arxiv_labels.append(category_class_id)


    max_sim_model = MaxSimClassifier("wiki", best_hparams['preprocess'], best_hparams['vector_size'],best_hparams['min_count'],
                                    best_hparams['epochs'],best_hparams['dm'],best_hparams['window'],workers=8)

    max_sim_model.fit_articles(x_arxiv_papers, y_arxiv_labels)

    top2_acc = max_sim_model.score(x_test_CLF_A_full, y_test_CLF_A_full,eval="top2") 
    top1_acc = max_sim_model.score(x_test_CLF_A_full, y_test_CLF_A_full,eval="top1")

    print("Run {}. MSC Top-1 Acc: {:.4f}, Top-2 Acc.:{:.4f}".format(i, top1_acc, top2_acc))
    top2_acc_list.append(top2_acc)
    top1_acc_list.append(top1_acc)

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


Run 0. MSC Top-1 Acc: 0.1375, Top-2 Acc.:0.2687
Run 1. MSC Top-1 Acc: 0.2112, Top-2 Acc.:0.3837
Run 2. MSC Top-1 Acc: 0.2100, Top-2 Acc.:0.3663
Run 3. MSC Top-1 Acc: 0.1837, Top-2 Acc.:0.3100
Run 4. MSC Top-1 Acc: 0.1837, Top-2 Acc.:0.3337
Run 5. MSC Top-1 Acc: 0.2000, Top-2 Acc.:0.3137
Run 6. MSC Top-1 Acc: 0.2050, Top-2 Acc.:0.3038
Run 7. MSC Top-1 Acc: 0.1837, Top-2 Acc.:0.3475
Run 8. MSC Top-1 Acc: 0.1625, Top-2 Acc.:0.3025
Run 9. MSC Top-1 Acc: 0.2062, Top-2 Acc.:0.3162
Run 10. MSC Top-1 Acc: 0.1950, Top-2 Acc.:0.3375
Run 11. MSC Top-1 Acc: 0.1650, Top-2 Acc.:0.3088
Run 12. MSC Top-1 Acc: 0.1888, Top-2 Acc.:0.2963
Run 13. MSC Top-1 Acc: 0.1713, Top-2 Acc.:0.3150
Run 14. MSC Top-1 Acc: 0.2362, Top-2 Acc.:0.3713
Run 15. MSC Top-1 Acc: 0.1600, Top-2 Acc.:0.3575
Run 16. MSC Top-1 Acc: 0.1475, Top-2 Acc.:0.2462
Run 17. MSC Top-1 Acc: 0.1975, Top-2 Acc.:0.3125
Run 18. MSC Top-1 Acc: 0.2288, Top-2 Acc.:0.3675
Run 19. MSC Top-1 Acc: 0.2100, Top-2 Acc.:0.3625
Run 20. MSC Top-1 Acc: 0.2425,

In [ ]:
print("Avg Top-2 acc:{:.4f}+-{:.4f}".format(np.mean(np.array(top2_acc_list)),np.std(np.array(top2_acc_list))))
print("Avg Top-1 acc:{:.4f}+-{:.4f}".format(np.mean(np.array(top1_acc_list)),np.std(np.array(top1_acc_list))))

Avg Top-2 acc:0.3310+-0.0332
Avg Top-1 acc:0.1957+-0.0255


In [ ]:
x_arxiv_papers

["Mechanism to Mitigate AVX-Induced Frequency Reduction. Modern Intel CPUs reduce their frequency when executing wide vector operations (AVX2 and AVX-512 instructions), as these instructions increase power consumption. The frequency is only increased again two milliseconds after the last code section containing such instructions has been executed in order to prevent excessive numbers of frequency changes. Due to this delay, intermittent use of wide vector operations can slow down the rest of the system significantly. For example, previous work has shown the performance of web servers to be reduced by up to 10% if the SSL library uses AVX-512 vector instructions. These performance variations are hard to predict during software development as the performance impact of vectorization depends on the specific workload. We describe a mechanism to reduce the slowdown caused by wide vector instructions without requiring extensive changes to existing software. Our design allows the developer to 